In [33]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [105]:
data = pd.read_csv("D:\\food recommendation system\\cuisines.csv")
df = data.copy()
df.head()

,name,image_url,description,cuisine,course,diet,prep_time,ingredients,instructions
0,Thayir Semiya Recipe (Curd Semiya),https://www.archanaskitchen.com/images/archana...,Thayir Semiya or Curd Vermicelli is a quick di...,Indian,Lunch,Vegetarian,Total in 35 M,\n\n\t\t\t\t\t\t\t \t\t1/2 cup Semiya (Vermice...,"To begin making the Thayir Semiya recipe, firs..."
1,Chettinad Style Kara Kuzhambu Recipe with Pota...,https://www.archanaskitchen.com/images/archana...,Chettinad Style Kara Kuzhambu Recipe with Pot...,South Indian Recipes,Lunch,Vegetarian,Total in 75 M,\nFor ground masala\n\n\t\t\t\t\t\t\t \t\t1/4 ...,To begin making the Chettinad Style Kara Kuzha...
2,Goan Chana Ros Recipe (White Peas Curry),https://www.archanaskitchen.com/images/archana...,Goan Chana Ros is a vegetarian dish from Goan ...,Goan Recipes,Lunch,Vegetarian,Total in 530 M,\n\n\t\t\t\t\t\t\t \t\t1/2 cup Dried Green Pea...,"To prepare Goan Chana Ros recipe, soak the whi..."
3,Minced Meat And Egg Croquettes Recipe,https://www.archanaskitchen.com/images/archana...,The croquette is usually cigar shaped or cylin...,North Indian Recipes,Appetizer,Non Vegeterian,Total in 55 M,\n\n\t\t\t\t\t\t\t \t\t500 grams Mutton\t\t\t\...,To begin making the Minced Meat And Egg Croque...
4,Thekera Tenga Recipe,https://www.archanaskitchen.com/images/archana...,Thekera Tenga is a Sour curry from Assamese cu...,Assamese,Dinner,Vegetarian,Total in 30 M,\n\n\t\t\t\t\t\t\t \t\t4 Potatoes (Aloo)\t\t\t...,"To begin making the Thekera Tenga recipe, Heat..."


In [106]:
df.isnull().sum()

name             0
image_url        0
description      0
cuisine          6
course          38
diet            37
prep_time       10
ingredients      0
instructions     0
dtype: int64

In [107]:
df = df.dropna()
df.isnull().sum()

name            0
image_url       0
description     0
cuisine         0
course          0
diet            0
prep_time       0
ingredients     0
instructions    0
dtype: int64

In [109]:

df['ingredients'].replace('\t', '', regex=True, inplace=True)
df['ingredients'].replace('\n', '', regex=True, inplace=True)
df['ingredients'].replace('[0-9]+', '', regex=True, inplace=True)
df['ingredients'].replace('[^\w\s]', '', regex=True, inplace=True)
df['ingredients'].replace('cup', '', regex=True, inplace=True)
df['ingredients'].replace('teaspoon', '', regex=True, inplace=True)
df['ingredients'].replace('pinch', '', regex=True, inplace=True)



In [112]:
def separate_words_by_commas(text):
    words = text.split()
    return ', '.join(words)
df['ingredients'] = df['ingredients'].apply(separate_words_by_commas)
df.head()

,name,image_url,description,cuisine,course,diet,prep_time,ingredients,instructions
0,Thayir Semiya Recipe (Curd Semiya),https://www.archanaskitchen.com/images/archana...,Thayir Semiya or Curd Vermicelli is a quick di...,Indian,Lunch,Vegetarian,Total in 35 M,"Semiya, Vermicelli, roasted, Curd, Dahi, Yogur...","To begin making the Thayir Semiya recipe, firs..."
1,Chettinad Style Kara Kuzhambu Recipe with Pota...,https://www.archanaskitchen.com/images/archana...,Chettinad Style Kara Kuzhambu Recipe with Pot...,South Indian Recipes,Lunch,Vegetarian,Total in 75 M,"For, ground, masala, Methi, Seeds, Fenugreek, ...",To begin making the Chettinad Style Kara Kuzha...
2,Goan Chana Ros Recipe (White Peas Curry),https://www.archanaskitchen.com/images/archana...,Goan Chana Ros is a vegetarian dish from Goan ...,Goan Recipes,Lunch,Vegetarian,Total in 530 M,"Dried, Green, Peas, Vatana, soaked, overnight,...","To prepare Goan Chana Ros recipe, soak the whi..."
3,Minced Meat And Egg Croquettes Recipe,https://www.archanaskitchen.com/images/archana...,The croquette is usually cigar shaped or cylin...,North Indian Recipes,Appetizer,Non Vegeterian,Total in 55 M,"grams, Mutton, minced, Whole, Eggs, boiled, an...",To begin making the Minced Meat And Egg Croque...
4,Thekera Tenga Recipe,https://www.archanaskitchen.com/images/archana...,Thekera Tenga is a Sour curry from Assamese cu...,Assamese,Dinner,Vegetarian,Total in 30 M,"Potatoes, Aloo, thinly, sliced, tablespoon, Mu...","To begin making the Thekera Tenga recipe, Heat..."


In [113]:
df['tags'] = df['instructions'].apply(separate_words_by_commas)
df.head()

,name,image_url,description,cuisine,course,diet,prep_time,ingredients,instructions,tags
0,Thayir Semiya Recipe (Curd Semiya),https://www.archanaskitchen.com/images/archana...,Thayir Semiya or Curd Vermicelli is a quick di...,Indian,Lunch,Vegetarian,Total in 35 M,"Semiya, Vermicelli, roasted, Curd, Dahi, Yogur...","To begin making the Thayir Semiya recipe, firs...","To, begin, making, the, Thayir, Semiya, recipe..."
1,Chettinad Style Kara Kuzhambu Recipe with Pota...,https://www.archanaskitchen.com/images/archana...,Chettinad Style Kara Kuzhambu Recipe with Pot...,South Indian Recipes,Lunch,Vegetarian,Total in 75 M,"For, ground, masala, Methi, Seeds, Fenugreek, ...",To begin making the Chettinad Style Kara Kuzha...,"To, begin, making, the, Chettinad, Style, Kara..."
2,Goan Chana Ros Recipe (White Peas Curry),https://www.archanaskitchen.com/images/archana...,Goan Chana Ros is a vegetarian dish from Goan ...,Goan Recipes,Lunch,Vegetarian,Total in 530 M,"Dried, Green, Peas, Vatana, soaked, overnight,...","To prepare Goan Chana Ros recipe, soak the whi...","To, prepare, Goan, Chana, Ros, recipe,, soak, ..."
3,Minced Meat And Egg Croquettes Recipe,https://www.archanaskitchen.com/images/archana...,The croquette is usually cigar shaped or cylin...,North Indian Recipes,Appetizer,Non Vegeterian,Total in 55 M,"grams, Mutton, minced, Whole, Eggs, boiled, an...",To begin making the Minced Meat And Egg Croque...,"To, begin, making, the, Minced, Meat, And, Egg..."
4,Thekera Tenga Recipe,https://www.archanaskitchen.com/images/archana...,Thekera Tenga is a Sour curry from Assamese cu...,Assamese,Dinner,Vegetarian,Total in 30 M,"Potatoes, Aloo, thinly, sliced, tablespoon, Mu...","To begin making the Thekera Tenga recipe, Heat...","To, begin, making, the, Thekera, Tenga, recipe..."


In [114]:
df['cuisine'].describe()

count                     4179
unique                      54
top       North Indian Recipes
freq                       817
Name: cuisine, dtype: object

In [188]:
df['upvotes'] = 0
df.head()

,name,image_url,description,cuisine,course,diet,prep_time,ingredients,instructions,tags,upvotes
0,Thayir Semiya Recipe (Curd Semiya),https://www.archanaskitchen.com/images/archana...,Thayir Semiya or Curd Vermicelli is a quick di...,Indian,Lunch,Vegetarian,Total in 35 M,"Semiya, Vermicelli, roasted, Curd, Dahi, Yogur...","To begin making the Thayir Semiya recipe, firs...","To, begin, making, the, Thayir, Semiya, recipe...",0
1,Chettinad Style Kara Kuzhambu Recipe with Pota...,https://www.archanaskitchen.com/images/archana...,Chettinad Style Kara Kuzhambu Recipe with Pot...,South Indian Recipes,Lunch,Vegetarian,Total in 75 M,"For, ground, masala, Methi, Seeds, Fenugreek, ...",To begin making the Chettinad Style Kara Kuzha...,"To, begin, making, the, Chettinad, Style, Kara...",0
2,Goan Chana Ros Recipe (White Peas Curry),https://www.archanaskitchen.com/images/archana...,Goan Chana Ros is a vegetarian dish from Goan ...,Goan Recipes,Lunch,Vegetarian,Total in 530 M,"Dried, Green, Peas, Vatana, soaked, overnight,...","To prepare Goan Chana Ros recipe, soak the whi...","To, prepare, Goan, Chana, Ros, recipe,, soak, ...",0
3,Minced Meat And Egg Croquettes Recipe,https://www.archanaskitchen.com/images/archana...,The croquette is usually cigar shaped or cylin...,North Indian Recipes,Appetizer,Non Vegeterian,Total in 55 M,"grams, Mutton, minced, Whole, Eggs, boiled, an...",To begin making the Minced Meat And Egg Croque...,"To, begin, making, the, Minced, Meat, And, Egg...",0
4,Thekera Tenga Recipe,https://www.archanaskitchen.com/images/archana...,Thekera Tenga is a Sour curry from Assamese cu...,Assamese,Dinner,Vegetarian,Total in 30 M,"Potatoes, Aloo, thinly, sliced, tablespoon, Mu...","To begin making the Thekera Tenga recipe, Heat...","To, begin, making, the, Thekera, Tenga, recipe...",0


In [196]:
def increment(recipe):
    row_index = df[df['name'] == recipe].index[0]
    df.loc[row_index, 'upvotes'] += 1
    print(f'Upvoted {recipe}')

In [198]:
increment('Thayir Semiya Recipe (Curd Semiya)')
df.head()

Upvoted Thayir Semiya Recipe (Curd Semiya)


,name,image_url,description,cuisine,course,diet,prep_time,ingredients,instructions,tags,upvotes
0,Thayir Semiya Recipe (Curd Semiya),https://www.archanaskitchen.com/images/archana...,Thayir Semiya or Curd Vermicelli is a quick di...,Indian,Lunch,Vegetarian,Total in 35 M,"Semiya, Vermicelli, roasted, Curd, Dahi, Yogur...","To begin making the Thayir Semiya recipe, firs...","To, begin, making, the, Thayir, Semiya, recipe...",3
1,Chettinad Style Kara Kuzhambu Recipe with Pota...,https://www.archanaskitchen.com/images/archana...,Chettinad Style Kara Kuzhambu Recipe with Pot...,South Indian Recipes,Lunch,Vegetarian,Total in 75 M,"For, ground, masala, Methi, Seeds, Fenugreek, ...",To begin making the Chettinad Style Kara Kuzha...,"To, begin, making, the, Chettinad, Style, Kara...",0
2,Goan Chana Ros Recipe (White Peas Curry),https://www.archanaskitchen.com/images/archana...,Goan Chana Ros is a vegetarian dish from Goan ...,Goan Recipes,Lunch,Vegetarian,Total in 530 M,"Dried, Green, Peas, Vatana, soaked, overnight,...","To prepare Goan Chana Ros recipe, soak the whi...","To, prepare, Goan, Chana, Ros, recipe,, soak, ...",0
3,Minced Meat And Egg Croquettes Recipe,https://www.archanaskitchen.com/images/archana...,The croquette is usually cigar shaped or cylin...,North Indian Recipes,Appetizer,Non Vegeterian,Total in 55 M,"grams, Mutton, minced, Whole, Eggs, boiled, an...",To begin making the Minced Meat And Egg Croque...,"To, begin, making, the, Minced, Meat, And, Egg...",0
4,Thekera Tenga Recipe,https://www.archanaskitchen.com/images/archana...,Thekera Tenga is a Sour curry from Assamese cu...,Assamese,Dinner,Vegetarian,Total in 30 M,"Potatoes, Aloo, thinly, sliced, tablespoon, Mu...","To begin making the Thekera Tenga recipe, Heat...","To, begin, making, the, Thekera, Tenga, recipe...",0


In [199]:
df['prep_time'].replace('Total in', '', regex=True, inplace=True)
df['prep_time'].replace('M', '', regex=True, inplace=True) 
df.head()

,name,image_url,description,cuisine,course,diet,prep_time,ingredients,instructions,tags,upvotes
0,Thayir Semiya Recipe (Curd Semiya),https://www.archanaskitchen.com/images/archana...,Thayir Semiya or Curd Vermicelli is a quick di...,Indian,Lunch,Vegetarian,35,"Semiya, Vermicelli, roasted, Curd, Dahi, Yogur...","To begin making the Thayir Semiya recipe, firs...","To, begin, making, the, Thayir, Semiya, recipe...",3
1,Chettinad Style Kara Kuzhambu Recipe with Pota...,https://www.archanaskitchen.com/images/archana...,Chettinad Style Kara Kuzhambu Recipe with Pot...,South Indian Recipes,Lunch,Vegetarian,75,"For, ground, masala, Methi, Seeds, Fenugreek, ...",To begin making the Chettinad Style Kara Kuzha...,"To, begin, making, the, Chettinad, Style, Kara...",0
2,Goan Chana Ros Recipe (White Peas Curry),https://www.archanaskitchen.com/images/archana...,Goan Chana Ros is a vegetarian dish from Goan ...,Goan Recipes,Lunch,Vegetarian,530,"Dried, Green, Peas, Vatana, soaked, overnight,...","To prepare Goan Chana Ros recipe, soak the whi...","To, prepare, Goan, Chana, Ros, recipe,, soak, ...",0
3,Minced Meat And Egg Croquettes Recipe,https://www.archanaskitchen.com/images/archana...,The croquette is usually cigar shaped or cylin...,North Indian Recipes,Appetizer,Non Vegeterian,55,"grams, Mutton, minced, Whole, Eggs, boiled, an...",To begin making the Minced Meat And Egg Croque...,"To, begin, making, the, Minced, Meat, And, Egg...",0
4,Thekera Tenga Recipe,https://www.archanaskitchen.com/images/archana...,Thekera Tenga is a Sour curry from Assamese cu...,Assamese,Dinner,Vegetarian,30,"Potatoes, Aloo, thinly, sliced, tablespoon, Mu...","To begin making the Thekera Tenga recipe, Heat...","To, begin, making, the, Thekera, Tenga, recipe...",0


In [115]:
cuisines_all = df.cuisine
print(cuisines_all)
cuisines = cuisines_all.unique()
print(cuisines)

0                     Indian
1       South Indian Recipes
2               Goan Recipes
3       North Indian Recipes
4                   Assamese
                ...         
4231         Bengali Recipes
4232               Jharkhand
4233                  Indian
4234    North Indian Recipes
4235          Kerala Recipes
Name: cuisine, Length: 4179, dtype: object
['Indian' 'South Indian Recipes' 'Goan Recipes' 'North Indian Recipes'
 'Assamese' 'Tamil Nadu' 'Karnataka' 'Malvani' 'Kerala Recipes'
 'Gujarati Recipes\ufeff' 'Sindhi' 'Konkan' 'Hyderabadi' 'Andhra' 'Coorg'
 'Bengali Recipes' 'Mangalorean' 'Kongunadu' 'Punjabi' 'Udupi' 'Pakistani'
 'Chettinad' 'Rajasthani' 'Nepalese' 'Coastal Karnataka' 'Indo Chinese'
 'Parsi Recipes' 'Kashmiri' 'South Karnataka' 'Mughlai' 'Asian' 'Bihari'
 'Maharashtrian Recipes' 'North East India Recipes' 'Himachal'
 'Uttar Pradesh' 'Awadhi' 'Sichuan' 'North Karnataka' 'Middle Eastern'
 'Oriya Recipes' 'Uttarakhand-North Kumaon ' 'Lucknowi' 'Chinese'
 'Haryana

In [116]:
cuisines_frequency = {}

for cuisines in cuisines_all:
    if cuisines in cuisines_frequency:
        cuisines_frequency[cuisines] += 1
    else:
        cuisines_frequency[cuisines] = 1

print(cuisines_frequency)

{'Indian': 802, 'South Indian Recipes': 626, 'Goan Recipes': 83, 'North Indian Recipes': 817, 'Assamese': 26, 'Tamil Nadu': 158, 'Karnataka': 132, 'Malvani': 8, 'Kerala Recipes': 159, 'Gujarati Recipes\ufeff': 124, 'Sindhi': 32, 'Konkan': 31, 'Hyderabadi': 27, 'Andhra': 116, 'Coorg': 15, 'Bengali Recipes': 163, 'Mangalorean': 44, 'Kongunadu': 3, 'Punjabi': 85, 'Udupi': 12, 'Pakistani': 11, 'Chettinad': 67, 'Rajasthani': 116, 'Nepalese': 5, 'Coastal Karnataka': 14, 'Indo Chinese': 5, 'Parsi Recipes': 35, 'Kashmiri': 58, 'South Karnataka': 8, 'Mughlai': 31, 'Asian': 6, 'Bihari': 22, 'Maharashtrian Recipes': 136, 'North East India Recipes': 20, 'Himachal': 17, 'Uttar Pradesh': 11, 'Awadhi': 36, 'Sichuan': 1, 'North Karnataka': 14, 'Middle Eastern': 6, 'Oriya Recipes': 29, 'Uttarakhand-North Kumaon ': 4, 'Lucknowi': 9, 'Chinese': 2, 'Haryana': 3, 'Fusion': 16, 'Malabar': 11, 'Continental': 5, 'Jharkhand': 2, 'Sri Lankan': 8, 'Thai': 1, 'Arab': 1, 'Nagaland': 5, 'Afghan': 1}


In [182]:
diet = 'Vegetarian'
cuisine = 'Indian'
course = 'Lunch'
df_1 = df[(df['diet'] == diet) & (df['cuisine'] == cuisine) & (df['course'] == course)]
new_index = np.arange(len(df_1))
df_1.index = new_index
df_1.head()

,name,image_url,description,cuisine,course,diet,prep_time,ingredients,instructions,tags
0,Thayir Semiya Recipe (Curd Semiya),https://www.archanaskitchen.com/images/archana...,Thayir Semiya or Curd Vermicelli is a quick di...,Indian,Lunch,Vegetarian,Total in 35 M,"Semiya, Vermicelli, roasted, Curd, Dahi, Yogur...","To begin making the Thayir Semiya recipe, firs...","To, begin, making, the, Thayir, Semiya, recipe..."
1,Spicy Cabbage Rice Recipe (South Indian Style ...,https://www.archanaskitchen.com/images/archana...,Spicy Cabbage Rice takes very few ingredients ...,Indian,Lunch,Vegetarian,Total in 65 M,"s, Cooked, rice, Ghee, or, oil, as, required, ...","To prepare Spicy Cabbage Rice recipe, firstly ...","To, prepare, Spicy, Cabbage, Rice, recipe,, fi..."
2,No Onion No Garlic Aloo Makhana Sabzi Recipe,https://www.archanaskitchen.com/images/archana...,"Be it Navratri or Shravan maas, many practice ...",Indian,Lunch,Vegetarian,Total in 45 M,"Baby, Potatoes, Phool, Makhana, Lotus, Seeds, ...",To begin making the No Onion No Garlic Aloo Ma...,"To, begin, making, the, No, Onion, No, Garlic,..."
3,तुरई चना सब्ज़ी - Ridge Gourd With Black Gram D...,https://www.archanaskitchen.com/images/archana...,तुरई चना सब्ज़ी एक बहुत ही सरल सब्ज़ी है जिसे उत...,Indian,Lunch,Vegetarian,Total in 30 M,"कप, कल, चन, तरई, बड़, चमचच, सरस, क, तल, छट, चमच...","तुरई चना सब्ज़ी बनाने के लिए, सबसे पहले काले चन...","तुरई, चना, सब्ज़ी, बनाने, के, लिए,, सबसे, पहले,..."
4,Buckwheat Vegetable Khichdi Recipe,https://www.archanaskitchen.com/images/archana...,Eating whole grain and ancient grains is helpi...,Indian,Lunch,Vegetarian,Total in 150 M,"Buckwheat, Flour, Kuttu, Ka, Atta, Yellow, Moo...",To begin making the Buckwheat Vegetable Khichd...,"To, begin, making, the, Buckwheat, Vegetable, ..."


In [166]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_1['ingredients'])
user_input = input("Enter your preferred ingredients (separated by commas): ")
user_input = user_input.split(',')
user_input_vector = tfidf_vectorizer.transform([' '.join(user_input)])
similarities = cosine_similarity(user_input_vector, tfidf_matrix)
top_n = 10
top_indices = similarities.argsort()[0, ::-1][:top_n]
print("Recommended Recipes:")
result_1 = []
for idx in top_indices:
    print(df_1['name'].iloc[idx])
    result_1.append(df_1['name'].iloc[idx])

Enter your preferred ingredients (separated by commas): onion
Recommended Recipes:
Lauki Aloo Sabzi Recipe - Bottle Gourd Potato Vegetable Curry
Meat Masala Stuffed Bharwa Baingan Recipe
One Pot Methi Mushroom Brown Rice Pulao Recipe
Pudina Quinoa Recipe
Gajar Methi Sabzi Recipe (Carrot Fenugreek Leaves Stir Fry)
Quick Pulao With Onions And Tomatoes Recipe
Spicy Cabbage Rice Recipe (South Indian Style Mutta Kose Sadham Recipe)
Cabbage Carrot Onion Pudina Thepla Recipe
Spicy Rice Stuffed Paratha Recipe
Moong Sprouts Pulao Recipe


In [178]:
cv = CountVectorizer(max_features = 50000)
vectors = cv.fit_transform(df_1['ingredients']).toarray()
similarity = cosine_similarity(vectors)
similarity.shape

(184, 184)

In [200]:
def recommend(recipe):
    if not df_1[df_1['name'] == recipe].empty:
        index = df_1[df_1['name'] == recipe].index[0]
        distances = sorted(enumerate(similarity[index]), key=lambda x: x[1], reverse=True)
        similar_recipes = []
        for i, _ in distances[1:10]:
            similar_recipes.append(df_1.iloc[i]['name'])
        return similar_recipes
    else:
        return f"Recipe '{recipe}' not found in the dataset."




# def recommend(recipe):
#     index = df_1[df_1['name'] == recipe].index[0]
#     distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
#     for i in distances[1:11]:
#         print(df_1.iloc[i[0]].name)

In [201]:
recommend('Pudina Quinoa Recipe')

['Tofu Green Pulao Recipe',
 'Green Moong Dal Pulao With Vegetables Recipe',
 'Sarson Saag Pulao Recipe-Spinach Mustard Greens Pulao',
 'Aromatic Vegetable Pulao Recipe',
 'Barnyard Millet Mushroom Biryani Recipe',
 'Broccoli Capsicum Pulao Recipe',
 'Beans And Carrot Masala Pulav With Black Eyed Beans Recipe',
 'Peppery Mushroom Biryani Recipe',
 'Baby Potatoes Pulao Recipe - Aloo Pulao Recipe']